In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "6789"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.readAll({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')

#Adding in Grazioso Salvare’s logo
image_filename = 'Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Header("John Sporn"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    # Buttons for filtering data sets
    html.Div(
        dcc.Dropdown(
            id = "Dropdown",
            # Values refering to specified areas in database
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain or Wilderness', 'value': 'mtn'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            placeholder = "Select a rescue type", # Place holder for dropdown menu
            multi = False, # can't select multiple values
            clearable = False, # Menu will always have a value in place. Never will be cleared
            style = {"width" : "50%"} # Drop down menu will only drop down half of the page
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Features for datatable
        editable = False,
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = 'single',
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10
    ),
    html.Br(),
    html.Hr(),
    #Sets up the dashboard so that the pie chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                dcc.Graph(
                id='graph-id',
                className='col s12 m6',
                
                ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('Dropdown', 'value')])

def update_output(value):
    #Assigning each value to refer to specific data in the database
    if (value =='water'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {"$in": ["Chesapeake Bay Retriever", "Labrador Retriever Mix", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female", 
            "age_upon_outcome_in_weeks": {"$gte": 26.0}, 
            "$and": [
            {"age_upon_outcome_in_weeks": {"$lte": 156.0}}]
        }
        )))
    elif (value == 'mtn'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky",
                              "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0},
            "$and": [
            {"age_upon_outcome_in_weeks": {"$lte": 156.0}}]
        }
        )))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound",
                              "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0},
            "$and": [
                {"age_upon_outcome_in_weeks": {"$lte": 300.0}}]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")])

def update_graphs(viewData):
    # Pie chart for data 
    dff = pd.DataFrame.from_records(viewData) # Creating copy of dataframe
    
    figure = px.pie(data_frame = dff,
                    names = 'breed')
    
    return figure


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)

def update_map(viewData, derived_virtual_selected_rows):
    
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    
    animalChoice = None
    
    if not derived_virtual_selected_rows:
        animalChoice = dff.iloc[0]
    else:
        animalChoice = dff.iloc[derived_virtual_selected_rows[0]]
    
    # Setting variables to refer to correct row of data table
    lat = animalChoice[12]
    long = animalChoice[13]
    breed = animalChoice[3]
    animal_name = animalChoice[8]
    animal_sex_outcome = animalChoice[11]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1(animal_name),
                    html.P(animal_sex_outcome)
                ])
            ])
        ])
    ]


app